In [1]:
#Getting the lib
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import seaborn as sns

In [2]:
#Website
first_url = "https://bygningsreglementet.dk/Administrative-bestemmelser/Krav"
#Using library requests to download index.html (Response=200 is good)
response = requests.get(first_url)
#Storing the html code
html = response.content
#Using Beautiful soup to parse the html for pythonformat
soup = BeautifulSoup(html, 'html.parser')

In [3]:
main_urls=[]
#Alle sider
for link in soup.nav.find_all("a", {"class": "link-square"}):
  main_urls.append(link["href"])
print("generated a list wiht", len(main_urls), "urls")

generated a list wiht 43 urls


In [4]:
#there are hidden links which we dont need
useful_urls=[]
for i in range(21):
    useful_urls.append(main_urls[i])
print("generated a list wiht", len(useful_urls), "urls")

generated a list wiht 21 urls


In [5]:
all_urls=[]
#Alle JS expanded pages
for link in useful_urls:
    response_link = requests.get(link)
    html_link = response_link.content
    soup_link = BeautifulSoup(html_link, 'html.parser')
    for paragraf in soup_link.find_all("a", {"class": "accordion__trigger"}):
        all_urls.append(paragraf["href"])

print("generated a list wiht", len(all_urls), "urls")

generated a list wiht 134 urls


In [32]:
paragraf=[]
#Alle paragraffer
for link in all_urls:
    response_link = requests.get(link)
    html_link = response_link.content
    soup_link = BeautifulSoup(html_link, 'html.parser')

    for i in soup_link.find("div", {"class": "accordion__content"}).find_all("div", {"class": "accordion__tag"}):
        if i.text.strip()!="":
            paragraf.append(i.text.strip())
        elif i.text.strip()=="" and i.div!=None and i.parent.parent.parent.find("div", {"class": "accordion__tag"}).text.strip()!="" and i.parent["class"]!=['accordion__row', 'accordion__row--footnote-list']:
            paragraf.append(i.parent.parent.parent.find("div", {"class": "accordion__tag"}).text.strip())



print("generated a list with", len(paragraf), "paragraffer")

generated a list with 429 paragraffer


In [27]:
#TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST TEST 


link="https://bygningsreglementet.dk/Tekniske-bestemmelser/03/Krav/63_68#cb9094f7-1f6c-44c2-8fe0-40ab46bc48fb"
response_link = requests.get(link)
html_link = response_link.content
soup_link = BeautifulSoup(html_link, 'html.parser')


"Webshop.ds.dk" not in soup_link.find("div", {"class": "accordion__content"}).find_all("div", {"class": "accordion__content"})[1].text



True

In [30]:
beskrivelse=[]
#Alle Beskrivelser
for link in all_urls:
    response_link = requests.get(link)
    html_link = response_link.content
    soup_link = BeautifulSoup(html_link, 'html.parser')

    for i in soup_link.find("div", {"class": "accordion__content"}).find_all("div", {"class": "accordion__content"}):
        if "Se vejledningen her" not in i.text.strip() and "Anvendelsesområde" not in i.text and "Webshop.ds.dk" not in i.text and "hjemmeside" not in i.text and i.parent["class"]!=['accordion__row', 'accordion__row--footnote-list']:
            beskrivelse.append(i.text.strip())

print("generated a list with", len(beskrivelse), "beskrivelser")

# Class=accordion__content  id=0e7e01f8-cc86-44b3-a046-1f4663536d54 er splittet op i to
# Prøv at sortere for id - DS reklamer har ikke et id men det har de andre!

generated a list with 459 beskrivelser


In [ ]:
#Laver en dataframe med al data
br18_df=pd.DataFrame({'paragraf': paragraf,'beskrivelse': beskrivelse})
br18_df=br18_df.set_index("paragraf")
br18_df

In [33]:
test=pd.DataFrame(beskrivelse)
test.to_excel("test.xlsx")
test2=pd.DataFrame(paragraf)
test2.to_excel("test2.xlsx")